In [1]:
import numpy as np
import random as rd
from time import process_time

flow = np.array([
   [0, 5, 2,  4, 1],
   [5, 0, 3, 0, 2],
   [2, 3, 0, 0, 0],
   [4, 0, 0, 0, 5],
   [1, 2, 0, 5, 0]
])

dist = np.array([
   [0, 1, 1, 2, 3],
   [1, 0, 2, 1, 2],
   [1, 2, 0, 1, 2],
   [2, 1, 1, 0, 1],
   [3, 2, 2, 1, 0]
])

In [2]:
def cost(flow):
    global dist                                  
    cost = flow*dist
    return np.sum(cost[np.triu_indices(5,1)])*2               # (sum of upper triangle of cost matrix)*2 
#print(flow[np.triu_indices(5,1)])
#print(np.sum(flow[np.triu_indices(5,1)]))

In [3]:
### INITIAL SOLUTION ###

def initialSol (mat):
    ndim = mat.shape[0]                                        # number of rows
    initial_sol = rd.sample(range(ndim), ndim)                 # Return the sampled list of unique random integers.
    return mat[np.ix_(initial_sol,initial_sol)], initial_sol   # first element holds updated flow matrix

s_zero = initialSol(flow)                                      # set the initial solution
print("Random initial solution: " + str(np.array(s_zero[1])+1))
print("Random initial solution's cost: " + str(cost(s_zero[0])))   

Random initial solution: [2 1 3 5 4]
Random initial solution's cost: 60


In [4]:
length = 5  # number of facilities/locations
N = 10      # size of neighboorhood matrix C(5,2)
neighbors = np.zeros((N, length +2), dtype=int)                 # neighboorhood matrix
#print (len(neighbors)) #length:10
#print (str(neighbors[idx, -2:]))

# 2 Opt Neighborhood 
def swap(sol_n):
    global a, neighbors
    for i in range (length):                                    # range (5): (0,5)
        j=i+1
        for j in range(length):
            if  i<j:
                a=a+1 
                sol_n[j], sol_n[i] = sol_n[i], sol_n[j]         #swap two elements 
                neighbors[a, :-2] = sol_n            
                neighbors[a, -2:] = [sol_n[i], sol_n[j]]
                sol_n[i], sol_n[j] = sol_n[j], sol_n[i]

"""sol_n= np.array(
   [2, 4, 0, 1, 3])              
sol_n[1], sol_n[0] = sol_n[0], sol_n[1] #swap two elements 
neighbors[0, :-2] = sol_n            
neighbors[0, -2:] = [sol_n[0], sol_n[1]]
print (str(neighbors))"""

def neighbor(sol):
    global neighbors, a
    a = -1
    swap(sol)                                                    # make a move to 2-opt neighboorhood 
    cost = np.zeros((len(neighbors)))                            # holds the cost of the neighbors: 1 rows 10 coloumns
    for b in range(len(neighbors)):
        cost[b] = solutionCost(neighbors[b, :-2])                # evaluate the cost of the candidate neighbors
    rank = np.argsort(cost)                                      # sorted index based on cost
    neighbors = neighbors[rank]                                  # sorted neighbor solutions
    k = []
    for n in range(N):
        k = rd.randint(0, N-1)
    return(neighbors[k,:-2])                                     # randomly select one of the neighbors

def solutionCost(solution):
    global flow, dist
    cost = flow*dist[np.ix_(solution,solution)]
    #return np.sum(cost[np.triu_indices(5,1)])*2
    return np.sum(cost)

#Acceptance Probability 
def acceptProb(delta, T):   
    return np.exp(-delta/T) > rd.random()                    

#SIMULATED ANNEALING

def simulatedAnnealing(flow, dist):
    current_sol  = s_zero[1]                                      #current solution <-- initial solution
    current_cost = cost(s_zero[0])                                #current solution cost <-- initial solution cost
    inc_sol  = s_zero[1]                                          #incumbent solution <-- initial solution
    inc_cost = cost(s_zero[0])                                    #incumbent solution cost <-- initial solution cost
    print("Initial Solution: " + str(np.array(inc_sol)+1))
    print("Initial Cost: " + str(inc_cost))
    T_zero =np.ceil( -0.5*current_cost/np.log(0.90))              # initial temperature
    T = T_zero                                                    # current temperature <-- initial temperature
    T_min = 0.001 
    alpha = 0.95
    mü = 5                                                        # number of iterations
    k = 0                                                         # iteration counter
    t_start = process_time()
    while T>T_min:
        for i in (0,mü):
            new_sol = neighbor(current_sol)
            new_cost = cost(flow[np.ix_(new_sol, new_sol)])
            if new_cost < current_cost | acceptProb(new_cost - current_cost, T):  # accept solution 
                current_sol  = new_sol                                            # update current solution
                current_cost = cost(flow[np.ix_(current_sol, current_sol)])       # update current cost
            if current_cost < inc_cost:                                                                         
                inc_sol = current_sol                                             # update incumbent solution
                inc_cost = cost(flow[np.ix_(inc_sol, inc_sol)])                   # update incumbent cost
        if k>15:                                                                  # start cooling after 15 iterations
            T = T*alpha       
        k = k+1                                                                   # update iteration counter
    t_finish = process_time()
    print("CPU time in seconds:", t_finish-t_start) 
    print("Final Simulated Annealing Solution: " + str(np.array(inc_sol)+1))
    print("Final Simulated Annealing Cost: " + str(inc_cost))
    print("Number of iterations: " + str(k))
    print("Initial temperature:" + str(T_zero))
    
# PRINT TO EXCEL 
    import xlwt

    x=str(np.array(s_zero[1])+1)
    y=str(cost(s_zero[0]))
    z=str(np.array(inc_sol)+1)
    w=str(inc_cost)
    p=t_finish-t_start
    q=str(k)
    f=str(T_zero)


    book = xlwt.Workbook(encoding="utf-8")

    sheet1 = book.add_sheet("Sheet 1")

    sheet1.write(0, 0, "Initial Solution:")
    sheet1.write(1, 0, "Initial Cost:")
    sheet1.write(2, 0, "Final Simulated Annealing Solution: ")
    sheet1.write(3, 0, "Final Simulated Annealing Cost: ")
    sheet1.write(4, 0, "CPU time in seconds:")
    sheet1.write(5, 0, "Number of iterations: ")
    sheet1.write(6, 0, "Initial temperature:")
    sheet1.write(0, 1, x)
    sheet1.write(1, 1, y)
    sheet1.write(2, 1, z)
    sheet1.write(3, 1, w)
    sheet1.write(4, 1, p)
    sheet1.write(5, 1, q)
    sheet1.write(6, 1, f)
    book.save("trial.xls")
    
    return str(np.array(inc_sol)+1), inc_cost

simulatedAnnealing(flow, dist)

Initial Solution: [2 1 3 5 4]
Initial Cost: 60
CPU time in seconds: 0.140625
Final Simulated Annealing Solution: [2 1 3 4 5]
Final Simulated Annealing Cost: 58
Number of iterations: 261
Initial temperature:285.0


('[2 1 3 4 5]', 58)